In [1]:
import os
import random
import pandas as pd
import numpy as np
import sklearn
from catboost import CatBoostRegressor, Pool
import catboost
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from datetime import datetime

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [3]:
train = pd.read_csv('preprocessing.csv')
test = pd.read_csv('../data/FlightDelay/test.csv')
submission = pd.read_csv('../data/FlightDelay/sample_submission.csv')

In [4]:
NaN_col = ['Origin_State', 'Destination_State', 'Airline', 'Estimated_Departure_Time', 'Estimated_Arrival_Time', 'Carrier_Code(IATA)', 'Carrier_ID(DOT)']

for col in NaN_col:
    mode = train[col].mode()[0]
    train[col] = train[col].fillna(mode)
    
    if col in test.columns:
        test[col] = test[col].fillna(mode)
print('Done.')

Done.


In [5]:
qual_col = ['Origin_Airport', 'Origin_State', 'Destination_Airport', 'Destination_State', 'Airline', 'Carrier_Code(IATA)', 'Tail_Number']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train[i])
    train[i] = le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test[i] = le.transform(test[i])
print('Done.')

Done.


In [6]:
train = train.dropna()

In [7]:
column_number = {}
for i, column in enumerate(submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train.loc[:, 'Delay_num'] = train['Delay'].apply(lambda x: to_number(x, column_number))
print('Done.')

train_x = train.drop(columns=['ID', 'Delay', 'Delay_num'])
train_y = train['Delay_num']
test_x = test.drop(columns=['ID'])

Done.


In [8]:
# regressor
# reg = catboost.CatBoostRegressor(iterations=500, learning_rate=0.05, loss_function='Logloss', verbose=0)
reg = catboost.CatBoostRegressor(iterations=500, learning_rate=0.05, verbose=0)

reg.fit(train_x, train_y, early_stopping_rounds=100)

In [9]:
pred = reg.predict(test_x, prediction_type='Probability')

In [10]:
for i in range(len(submission)):
    submission['Delayed'][i], submission['Not_Delayed'][i] = pred[i][0], pred[i][1]

/tmp/ipykernel_2247/1431158504.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['Delayed'][i], submission['Not_Delayed'][i] = pred[i][0], pred[i][1]


In [11]:
submission.to_csv('submission11.csv', index=False)